<a href="https://colab.research.google.com/github/melvinsackaria/DeepLearning/blob/main/Kaggle_Faker_News_Classifier_Using_LSTM_Deep_LEarning%7C_Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fake News Classifier Using Bidirectional LSTM**



```
# This is formatted as code
```


Dataset: https://www.kaggle.com/c/fake-news/data#

In [22]:
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [23]:
###Drop Nan Values
df=df.dropna()
## Get the Independent Features
X=df.drop('label',axis=1)
## Get the Dependent features
y=df['label']
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [24]:
X.shape

(18285, 4)

In [25]:
y.shape

(18285,)

In [26]:
import tensorflow as tf

In [27]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [28]:
### Vocabulary size
voc_size=5000

**Onehot Representation**


In [29]:
messages=X.copy()

In [30]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [31]:
messages.reset_index(inplace=True) # since nan values were dropped

In [32]:
import nltk
import re
from nltk.corpus import stopwords

In [33]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [35]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [36]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3690, 4962, 4838, 4134, 4360, 3432, 4243, 6, 671, 1012],
 [3096, 1174, 704, 3608, 2711, 529, 92],
 [4072, 1318, 4129, 2645],
 [2836, 1210, 2163, 1525, 4200, 2756],
 [360, 2711, 2242, 4129, 3428, 2897, 2711, 4973, 2060, 3940],
 [3692,
  3200,
  3094,
  3914,
  1972,
  781,
  3971,
  1573,
  1514,
  3709,
  1937,
  3738,
  1767,
  67,
  92],
 [988, 2768, 1455, 1713, 459, 182, 3992, 1188, 4224, 2188, 1424],
 [2942, 969, 1116, 4698, 4031, 4702, 781, 2407, 4224, 2188, 1424],
 [826, 290, 1045, 2522, 4950, 495, 1394, 1952, 781, 2365],
 [102, 4158, 4602, 396, 3944, 4697, 4424, 3694],
 [4452, 3410, 135, 2067, 3077, 1096, 3669, 1135, 4030, 3639, 1329],
 [1525, 4210, 4360, 495, 781, 4031],
 [1206, 4844, 2141, 281, 36, 164, 2963, 4008, 2022],
 [3958, 2651, 3812, 1260, 1482, 1329, 4131, 4224, 2188, 1424],
 [1692, 3763, 3222, 647, 4420, 4224, 2188, 1424],
 [4147, 4386, 4507, 2263, 916, 1904, 2260, 1230, 2482, 2972],
 [3431, 3140, 1174],
 [501, 2415, 2114, 2058, 781, 668, 3058, 92],
 [4583, 2435, 7

**Embedding Representation**


In [37]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...    6  671 1012]
 [   0    0    0 ... 2711  529   92]
 [   0    0    0 ... 1318 4129 2645]
 ...
 [   0    0    0 ... 4224 2188 1424]
 [   0    0    0 ... 4478 2334  139]
 [   0    0    0 ... 1727 1386 3907]]


In [38]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3690,
       4962, 4838, 4134, 4360, 3432, 4243,    6,  671, 1012], dtype=int32)

In [39]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))#1 lstm layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))#output layer - 1 neuron giving classification
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None



**Adding Dropout**




In [ ]:
## Creating model by adding drop out layer
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
len(embedded_docs),y.shape

(18285, (18285,))

In [41]:
#Final X and Y values
import numpy as np
X_final=np.array(embedded_docs) # x is present in embedded docs
y_final=np.array(y)

In [42]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

**Model Training**


In [45]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 37ms/step - loss: 0.3239 - accuracy: 0.8504 - val_loss: 0.1994 - val_accuracy: 0.9176
Epoch 2/10
192/192 [==============================] - 7s 34ms/step - loss: 0.1344 - accuracy: 0.9483 - val_loss: 0.1992 - val_accuracy: 0.9188
Epoch 3/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0938 - accuracy: 0.9652 - val_loss: 0.2171 - val_accuracy: 0.9132
Epoch 4/10
192/192 [==============================] - 8s 44ms/step - loss: 0.0715 - accuracy: 0.9740 - val_loss: 0.2493 - val_accuracy: 0.9157
Epoch 5/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0552 - accuracy: 0.9816 - val_loss: 0.2937 - val_accuracy: 0.9123
Epoch 6/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0437 - accuracy: 0.9861 - val_loss: 0.3621 - val_accuracy: 0.9118
Epoch 7/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0340 - accuracy: 0.9888 - val_loss: 0.4084 - val_accuracy: 0.910

**Performance Metrics And Accuracy**

https://stackoverflow.com/questions/68836551/keras-attributeerror-sequential-object-has-no-attribute-predict-classes

In [58]:
y_pred=(model.predict(X_test) > 0.5).astype("int32")
y_predClasses=np.argmax(y_pred,axis=1)

In [59]:
from sklearn.metrics import confusion_matrix

In [60]:
confusion_matrix(y_test,y_pred)

array([[3125,  294],
       [ 293, 2323]])

In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9027340513670257

In [62]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predClasses))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
